In [21]:
import tensorflow as tf
import scipy.io as sio
import cv2
import numpy as np
import os

In [11]:
def trainImage_generator(files, batch_size = 32, sz = (256, 256)):
  
  while True: 
    
    #extract a random batch 
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs and outputs 
    batch_x = []
    batch_y = []
    # batch_z = []
    
    
    for f in batch:




        #get the masks. Note that masks are png files 
        mask = cv2.imread(f'../downloads/data4test/aug_label/{f[:-3]}'+"png", cv2.IMREAD_GRAYSCALE).astype(np.float32)


        #preprocess the mask 
        # mask[mask >= 2] = 0 
        # mask[mask != 0 ] = 1
        


        #preprocess the raw images 
        # raw = Image.open(f'syntheticData/data/{f}')
        # raw = raw.resize(sz)
        # raw = np.array(raw)

        rawMat = sio.loadmat(f'../downloads/data4test/aug_data/{f}')
        xData = np.array(rawMat['vxSample'])
        yData = np.array(rawMat['vySample'])

        ImgSize = xData.shape

        input_image = np.stack((xData,yData,np.zeros(xData.shape)), -1)
        # input_mask = np.stack((mask,mask,mask), -1)


        # input_image = cv2.resize(input_image, sz)
        # input_mask = cv2.resize(mask, sz)

        # flipping random horizontal or vertical
        # if random.random() > 0.5:
        #     input_image = cv2.flip(input_image,0)
        #     input_mask = cv2.flip(input_mask,0)
        # if random.random() > 0.5:
        #     input_image = cv2.flip(input_image,1)
        #     input_mask = cv2.flip(input_mask,1)


        # input_image = np.asarray(input_image)
        # input_mask = np.asarray(mask)


        # input_mask = tf.image.rgb_to_grayscale(input_mask)

        # input_image = np.asarray(input_image)
        input_mask = mask


        input_mask[input_mask != 0 ] = 1

        batch_x.append(input_image)
        batch_y.append(input_mask)
        # batch_z.append(f)


    #preprocess a batch of images and masks 
    # batch_x = np.array(batch_x)/255.
    batch_x = np.array(batch_x)
    batch_y = np.array(batch_y)
    # batch_y = np.expand_dims(batch_y,3)

    # batch_x = preprocess_input(batch_x)
    # batch_y = preprocess_input(batch_y)



    yield (batch_x, batch_y)    

In [12]:
def validImage_generator(files, batch_size = 32, sz = (256, 256)):
  
  while True: 
    
    #extract a random batch 
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs and outputs 
    batch_x = []
    batch_y = []
    # batch_z = []
    
    
    for f in batch:



        #get the masks. Note that masks are png files 
        mask = cv2.imread(f'../downloads/data4test/label/{f[:-3]}'+"png", cv2.IMREAD_GRAYSCALE).astype(np.float32)


        #preprocess the mask 
        # mask[mask >= 2] = 0 
        # mask[mask != 0 ] = 1
        


        #preprocess the raw images 
        # raw = Image.open(f'syntheticData/data/{f}')
        # raw = raw.resize(sz)
        # raw = np.array(raw)

        rawMat = sio.loadmat(f'../downloads/data4test/data/{f}')
        xData = np.array(rawMat['vxSample'])
        yData = np.array(rawMat['vySample'])

        ImgSize = xData.shape

        input_image = np.stack((xData,yData,np.zeros(xData.shape)), -1)

        input_mask = mask



        input_image = np.asarray(input_image)
        input_mask = np.asarray(input_mask)
        input_mask[input_mask != 0 ] = 1

        batch_x.append(input_image)
        batch_y.append(input_mask)
        # batch_z.append(f)


    #preprocess a batch of images and masks 
    # batch_x = np.array(batch_x)/255.
    batch_x = np.array(batch_x)
    batch_y = np.array(batch_y)
    # batch_y = np.expand_dims(batch_y,3)

    # batch_x = preprocess_input(batch_x)



    yield (batch_x, batch_y)

In [20]:
%env SM_FRAMEWORK = tf.keras
import segmentation_models as sm

env: SM_FRAMEWORK=tf.keras


In [22]:
training_files = os.listdir("../downloads/data4test/aug_data/")
validate_files = os.listdir("../downloads/data4test/data/")
batch_size = 64
split = int(0.5 * len(training_files))
train_files = training_files[0:split]
test_files  = validate_files[split:]

train_data, valid_data = trainImage_generator(trainImage_generator(train_files, batch_size = batch_size)), validImage_generator(test_files, batch_size = batch_size)

In [26]:
BACKBONE = 'vgg16'
model = sm.Unet(BACKBONE, encoder_weights='imagenet')
    # model.compile('Adam', loss=sm.losses.bce_dice_loss, metrics=[sm.metrics.iou_score])
model.compile('Adam', loss=sm.losses.bce_dice_loss, metrics=["accuracy"])
train_steps = len(train_files) // batch_size
test_steps = len(test_files) // batch_size
model.fit(train_data,epochs = 32, steps_per_epoch = train_steps,validation_data = valid_data, validation_steps = test_steps, verbose = 1)


StopIteration: 